<a href="https://colab.research.google.com/github/magnolia2001/Forest_Estimation/blob/main/notebooks/NNandCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下是针对掩膜数据形状为(height, width)

如果掩膜数据形状为(1, height, width)则移除掩膜图像中的通道维度使其形状变为(height, width)

In [ ]:
import numpy as np
import pandas as pd
import datetime, os, cv2
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout, InputLayer, Flatten, Conv2D, MaxPool2D, AveragePooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint

# 制作标签数据和特征数据

# 窗口大小应为奇数，以保证标签在中间
size = 5 #define window size should be odd so that the label is in the middle
# 特征的形状，这里假设每个特征是一个大小为 (size, size) 的窗口，包含 90 个通道
shape = (90, size, size) #define shape of features
# np.ones(shape, dtype=None) 用于创建一个形状为 shape 的数组，并将所有元素初始化为 1.0
# 其中 shape：指定数组的形状，通常是一个整数或元组。 dtype：指定数组元素的数据类型（可选）。如果不指定，默认使用 float64 类型。
# labels1 用于存放标签数据（掩膜）, data1 用于存放提取的特征数据
# np.ones(1)返回的是一个只有一个元素的数组，其中该元素值为 1。
labels1 = np.ones(1) #array for labels
# 创建了一个数组，形状为 shape 即 (90, 5, 5) 的 NumPy 数组，并且所有的元素值都被初始化为 1.0 。
data1 = np.ones(shape) #array for features
# 扩展维度，便于后续拼接操作
data1 = np.expand_dims(data1, axis=0) #expand dimension to concatenate

# 遍历目录中的图像（假设有 20 张图像, 具体数量还需要根据自己的情况修改）
# 在 for j in range(20) 这个遍历过程中，区分 j < 10 和 j >= 10 的目的是为了处理不同的文件命名规则。
# 对于小于 10 的文件名，文件名是 "image_00X.npy"，其中 X 是单个数字（0 到 9）。
# 对于大于等于 10 的文件名，文件名是 "image_0XY.npy"，其中 XY 是两位数的数字（10 到 19）。
for j in range(20): #iterate over images in directory
  if j < 10:
    # 路径填写实际路径
    # 读取图像数据
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    # 读取掩膜数据
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像

    # 选择掩膜图像 y 中所有大于 0 的位置（即标签不为 0 的位置），并返回这些位置的索引。
    # indices 数组返回 N 个元素，其中 N 为掩膜图像 y 中所有大于 0 的元素个数。每个元素都是一个长度为 2 的行向量，表示符合条件元素的行列索引。
    # y > 0 是一个布尔条件，返回一个与 y 相同形状的布尔数组, 如果是大于 0，布尔值为 True，否则为 False。
    # np.argwhere() 是 NumPy 库中的一个函数，它返回数组中满足某个条件的所有索引（行列坐标），即满足条件的元素的坐标位置。
    # 这里 indices 是一个形状为 (N, 2) 的二维数组，每一行是 (y, x) 坐标. y 为行索引, x 为列索引
    indices = np.argwhere(y > 0) #select all values with label

    # indices_2d 是 indices 数组的一个切片，是一个 二维数组, 它包含了所有掩膜图像中标签值大于 0 的位置的 列索引。
    # 切片操作 indices[:, 1:] 就是提取所有行中的第二列（即 行 和 列 坐标中的 列索引）。
    # indices_2d = indices[:, 1:] #extract indices

    # 初始化一个数组 ind_y 来收集符合条件的标签位置。
    # np.ones(2) 会创建一个包含 2 个元素的数组，所有元素的值为 1. 。
    # .reshape(-1, 2) 将该数组的形状重塑为 (-1, 2)，表示按列数为 2 进行重塑，-1 表示自动计算行数。由于只有 2 个元素，这会将数组变成形状为 (1, 2) 的二维数组。
    ind_y = np.ones(2).reshape(-1,2) #array to collect indices

    # 遍历掩膜中的每个标签位置
    # for i in indices_2d: #iterate over indices
    for i in indices:
      # 提取图像块，并检查其形状。
      # size//2 表示 size 除以 2 的整数部分，用于确定图像块中心点到边界的距离。右端点的值之所以加 1 是因为区间是左闭右开的,所以加 1 保证能取到右端点.
      # 整个i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2)表达式计算出一个范围，用于选取以 (i[0], i[1])（标签的 y, x 坐标） 为中心，上下各延伸 size//2 个像素的区域。
      # i[0] 是当前标签位置的 y 坐标（行索引）。i[1] 是当前标签位置的 x 坐标（列索引）。
      # 利用 shape 确保当前窗口大小与指定的窗口大小一致
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape: #select only features with the same shape because of labels at the image border
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1] #save them temporary
        # 将 temp 的维度扩展一个维度，使得它变成一个形状为 (1, channels, size, size) 的四维数组。扩展维度的目的是为了能够将 temp 与其他提取的窗口进行拼接。
        temp2 = np.expand_dims(temp, axis=0) #expand dimension to concatenate
        # 拼接特征数据
        # data1 最终会变成 (num_samples, 90, 5, 5)，其中 num_samples 是提取的窗口数量。
        data1 = np.concatenate((data1, temp2), axis=0) #concatenation
        # 拼接标签索引
        # i.reshape(-1, 2) 会把 i 重新调整为一个形状为 (1, 2) 的二维数组
        # axis=0 表示在 第 0 维（行方向） 进行拼接，即新添加的行会被添加到原数组的最后。
        # ind_y 则是一个 (num_samples, 2) 的数组，每个样本对应一个标签位置的 (y, x) 坐标。
        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0) #concatenation of index so that they have the same order and length as the features

    # 去掉第一个虚拟值
    # 初始时，ind_y 中的第一个元素是 np.ones(2).reshape(-1,2) 创建的虚拟数据。此步骤是将它移除，只保留实际的标签坐标。
    ind_y = ind_y[1:] #remove first dummy values
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # 提取标签值
    # 从这句代码应该可以看出原作者的掩膜图像的形状包含了通道维度,即形状为(1, height, width)
    # data_y = y[0, indices_1, indices_2] #extract labels
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0) #concatenate labels

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像
    indices = np.argwhere(y > 0)
    # indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    # for i in indices_2d:
    for i in indices:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    # 去掉第一个虚拟值
    ind_y = ind_y[1:]
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # data_y = y[0, indices_1, indices_2]
    # 提取标签值
    # 每一对 (indices_1[i], indices_2[i]) 会自动匹配，得到对应位置的标签值。
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0)

# 移除第一个虚拟值
# data1 的形状会是 (num_samples, 90, 5, 5)，其中 num_samples 是提取的窗口数量（即符合条件的标签数量）。一个四维数组
# labels1 的形状会是 (num_samples,)，其中 num_samples 是所有图像中符合条件的标签数量。一个一维数组
data1 = data1[1:] #remove first dummy values
labels1 = labels1[1:] #remove first dummy values

# data1 和 labels1 应该是 一一对应的，因为它们的样本数（num_samples）相同。
# 获得标签数据和特征数据
features = data1
labels = labels1


### Optional: Create equally distributed data set

对数据进行采样，平衡各个标签类的样本数量。它通过将不同标签范围的样本分成多个区间（每3米为一个区间），并对每个区间进行随机抽样，最终创建一个平衡的数据集。

In [ ]:
# 每个类别的样本数，确保每个标签区间有 800 个样本
sample_size = 800 #every class with labels smaller 36 meters has over 800 values
#features = np.mean(features, axis=(2, 3)) # patch mean of size * size features

# 生成从 3 到 36 步长为 3 的数字列表，即 [3, 6, 9, ..., 36]
num = (list(range(3, 37, 3))) #create list from 3 to 36 step 3
# 假设每个样本是一个 5x5 的图像块（大小为 5x5，90 个通道）
shape = (90, 5, 5)
# 创建一个初始的数组用于存储特征数据，形状为 (90, 5, 5)
data_bal = np.ones(shape) #create array to fill with features
# 扩展维度，使得形状变为 (1, 90, 5, 5)，这样可以进行拼接
data_bal = np.expand_dims(data_bal, axis=0) #expand one dimension to concatenate
# 创建一个用于存储标签的初始数组，形状为 (1,)
data_lab = np.ones(1) #create array to fill labels

# 遍历每个标签区间
for i in num:
  # 注意这个 i 是区间右端点
  # 从标签中选择属于当前区间的索引
  # np.where() 返回的是一个元组，元组的元素个数取决于判断条件中的数据的维度, 元组的每个元素都是一个 数组，这些数组表示满足条件的元素在原始数组中的索引。因此，需要通过 indices[0] 访问索引数组
  # 如果输入数组是 多维的，返回的元组会包含 每一维的索引数组。例如，若数组是三维的，返回的元组就会包含三个数组，分别表示满足条件的元素在三维空间中每一维的索引。
  indices = np.where((labels > i-3) & (labels <= i)) #select indcies from every 3 meter interval until 36
  # 在当前区间中随机抽样 800 个样本
  # sampled_indices = np.random.choice(indices[0].flatten(), size=sample_size, replace=False) #random sample of each interval
  sampled_indices = np.random.choice(indices[0], size=sample_size, replace=False)
  # 提取对应的特征和标签
  tempx = features[sampled_indices]
  tempy = labels[sampled_indices]
  # 将当前区间的特征和标签拼接到平衡数组中
  data_bal = np.concatenate((data_bal, tempx), axis=0)
  data_lab = np.concatenate((data_lab, tempy), axis=0)

# 处理 labels > 36 的标签，这部分直接拼接
indices = np.where((labels > 36)) #add the values > 36 m, they are so few no sample needed
# sampled_indices = indices[0].flatten()
sampled_indices = indices[0]
tempx = features[sampled_indices]
tempy = labels[sampled_indices]
data_bal = np.concatenate((data_bal[1:], tempx), axis=0)
data_lab = np.concatenate((data_lab[1:], tempy), axis=0)

### Neural Network

如果你需要经过数据平衡之后的数据集应该在模型中使用特征数据 data_bal 和 标签数据 data_lab 来进数据集的和划分,而不是使用 features 和 labels 来进行数据集的划分(因为features 和 labels 未经过上采样)

### Training and evaluation

划分训练集和测试集

In [ ]:
# features 数组形状为 (num_samples, 90, 5, 5)，意味着每个样本有 90 个特征（或 90 个通道），每个特征是一个 5x5 的空间窗口。
# axis=(2, 3) 表示对每个特征的 5x5 窗口进行平均。结果是将每个 5x5 窗口压缩为一个标量（均值）
# features_mean 是一个二维数组，每个样本有 90 个特征（每个特征是原来 5x5 窗口的平均值）。
# 将每个特征的 5x5 窗口进行均值处理
features_mean = np.mean(features, axis=(2, 3)) # patch mean of size * size features
# 现在 features_mean 的形状是 (num_samples, 90)，适用于NN或者其他传统机器学习算法
# 注意: train_test_split 只能处理 NumPy 数组或 Pandas DataFrame，并不能直接处理 TensorFlow Dataset 对象。因此，这部分代码在处理 TensorFlow Dataset 时会出错。
X_train, X_test, y_train, y_test = train_test_split(features_mean, labels , test_size = 0.3, random_state=3)


In [ ]:
# %load_ext 是 Jupyter Notebook 中的一个魔法命令，用于加载扩展。
# tensorboard 是 TensorFlow 提供的一个工具，用于可视化训练过程中的数据，如损失函数、准确率、权重更新等。
# 在加载扩展后，你可以在 Jupyter Notebook 中直接使用 %tensorboard 命令来启动 TensorBoard。 在代码的 %tensorboard --logdir logs 部分启动了 TensorBoard
%load_ext tensorboard


神经网络模型定义

In [ ]:
# 使用 Sequential 模型，适用于线性堆叠的神经网络。
modelNn = Sequential()  # build neural network
# 第一层：Dense(128, input_shape=(10,), ...)代表输入层，输入形状为 90,)，表示每个输入样本有 90 个特征。这个 90 是硬编码的，但实际上它应该等于特征的维度数，这里可能需要根据实际情况进行修改。
# ernel_initializer='normal' 表示权重的初始化方式为正态分布
# activation='relu' 使用 ReLU 激活函数。
modelNn.add(Dense(128, input_shape=(90,), kernel_initializer='normal', activation='relu'))
# 第二层和第三层：分别包含 256 个神经元，也是全连接层，使用 ReLU 激活函数。
modelNn.add(Dense(256, kernel_initializer='normal', activation='relu'))
modelNn.add(Dense(256, kernel_initializer='normal', activation='relu'))
modelNn.add(Dense(128, kernel_initializer='normal', activation='relu'))
# 使用 Dropout(0.4) 防止过拟合，随机丢弃 40% 的神经元
modelNn.add(Dropout(0.4))
# 输出层包含 1 个神经元，activation='linear' 表示线性激活函数，适用于回归任务。
modelNn.add(Dense(1, kernel_initializer='normal', activation='linear'))

# 打印模型的结构，包括每一层的类型、输出形状、参数数量等信息，帮助你了解模型的结构。
modelNn.summary()


 模型编译

In [ ]:
# 损失函数：loss='mean_absolute_error' 使用绝对误差作为回归问题的损失函数。
# 优化器：optimizer='adam' 使用 Adam 优化器，它是目前常用的一种高效的优化算法。
# 评估指标：metrics=['mean_absolute_percentage_error'] 使用平均绝对百分比误差（MAPE）作为评估指标。

# 注意: 在训练过程中，优化算法会根据 val_loss 来更新模型的参数，因为 val_loss 是损失函数的值，而损失函数通常是模型优化的目标。
#    MAPE 作为评估指标，不会直接影响模型的参数更新，它仅用于评价模型在验证集上的相对误差，帮助你了解模型的实际表现。
modelNn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile model


回调函数设置

In [ ]:
# 生成一个包含当前时间戳的日志目录路径。
# os.path.join("logs", ...) 用于将 logs 目录和时间戳连接成完整的路径。
# datetime.datetime.now() 获取当前的日期和时间。
# .strftime("%Y%m%d-%H%M%S") 是日期时间格式化方法，将当前时间格式化为 YYYYMMDD-HHMMSS 形式的字符串（例如，20231230-140102），这样可以确保每次训练的日志目录都不同，避免覆盖。
# 结果是一个路径，如 logs/20231230-140102，这个路径会作为 TensorBoard 的日志目录，用于存储当前训练的所有日志文件。
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))  # log directory for tensorboard
# 可视化训练过程中的日志信息，logdir 为日志文件夹的路径。histogram_freq=1 表示每个 epoch 保存直方图。
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
# 模型保存, 仅保存表现最好的模型。
# 验证集的损失（val_loss） 是指模型在验证集上的损失函数的值。损失函数度量了模型在验证集上的预测与实际标签之间的差距。损失函数在训练过程中用来指导模型的优化过程。
# ModelCheckpoint 是 Keras 提供的一个回调函数，用于在训练过程中根据某个指定的条件保存模型。默认情况下，ModelCheckpoint 会监控模型的 val_loss（验证集的损失）。
# save_best_only=True 表示只有在验证集的性能（如验证损失或验证准确率,默认情况下是验证集的损失）比之前的最小值还小，才会保存模型的权重。否则，当前的模型会被忽略。
# 这个回调函数会在每个 epoch 结束后检查模型的性能，并根据设置的条件决定是否保存当前模型。
model_save = ModelCheckpoint("/content/gdrive/MyDrive/NNmodels/best_model3.hdf52", save_best_only=True)  # directory for best model


模型训练

In [ ]:
# 使用 model.fit() 训练模型：
# Xtrain 和 ytrain 是训练集的特征和标签。
# epochs=100 训练 100 轮。
# validation_data=(Xtest, ytest) 在每一轮训练后，使用测试集 Xtest 和 ytest 进行验证。
# allbacks=[tensorboard_callback, model_save] 传入回调函数，分别用于 TensorBoard 可视化和保存最好的模型。
modelNn.fit(Xtrain, ytrain, epochs = 100, validation_data=(Xtest, ytest), callbacks=[tensorboard_callback, model_save]) #fit model

加载最好的模型

In [ ]:
bmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_model3.hdf52') #load best model

 模型预测

In [ ]:
# 使用训练好的模型对测试集 Xtest 进行预测，返回预测值 ypred_nn。
ypred_nn = bmodel.predict(Xtest)


模型评估

In [ ]:
# 计算测试集的均方误差。 (MSE)
mse_nn = mse(ytest, ypred_nn)
# 计算均方根误差 (RMSE)
rmse_nn = mse_nn ** (1/2)
# 计算平均绝对误差 (MAE)
mae_nn = mae(ytest, ypred_nn)
# 平均绝对百分比误差 (MAPE)
mape_nn = mape(ytest, ypred_nn)

# 打印出 MAPE、MAE 和 RMSE 评估指标，帮助你评估模型的表现
print(mape_nn)
print(mae_nn)
print(rmse_nn)


In [ ]:
# 启动 TensorBoard 并将其指向训练日志目录（logs）。
# %tensorboard 是 Jupyter 魔法命令，用于启动 TensorBoard 可视化工具。
# --logdir 是一个命令行参数，用于指定 TensorBoard 要读取的日志文件夹。这个目录包含了你在训练过程中生成的所有日志文件，TensorBoard 会通过读取这些日志文件来展示模型训练的过程。
# logs 目录是用来存放训练过程中记录的所有日志文件的地方。通过使用 --logdir logs 参数，TensorBoard 会读取这个目录下的文件，从而显示出训练过程中的各种可视化图表。
%tensorboard --logdir logs

Quantile(分位数) 和统计信息

In [ ]:
# 计算预测结果的均值
mean_nn = np.mean(ypred_nn[:])  # calculate mean
# 计算预测结果的不同分位数（1%, 25%, 50%, 75%, 99%）
quantiles_nn = np.percentile(ypred_nn[:], [1, 25, 50, 75, 99])  # calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

# 计算标签的均值
mean_labels = np.mean(labels[:])
# 计算标签的不同分位数（1%, 25%, 50%, 75%, 99%）
quantiles_labels = np.percentile(labels[:], [1, 25, 50, 75, 99])

print(mean_nn)
print(quantiles_nn)
# 打印预测值中最小的 10 个和最大的 10 个。
print(np.sort(ypred_nn.flatten())[:10])  # print the 10 lowest predictions
print(np.sort(ypred_nn.flatten())[-10:][::-1])  # print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
# 打印标签中最小的 10 个和最大的 10 个。
print(np.sort(labels.flatten())[:10])
print(np.sort(labels.flatten())[-10:][::-1])
